Data Scrapping
https://medium.com/@sharuhinda/topsis-multi-criteria-decision-making-technique-542c362e768c


In [10]:
import numpy as np
print('numpy version:', np.__version__)

import pandas as pd
print('pandas version:', pd.__version__)

from bs4 import BeautifulSoup

from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium import webdriver

import sys
import pathlib

import logging

from time import sleep
import re

numpy version: 2.2.0
pandas version: 2.2.3


In [11]:
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

handler = logging.StreamHandler(stream=sys.stdout)
handler.setFormatter(logging.Formatter(fmt='[%(asctime)s: %(funcName)s: %(levelname)s] %(message)s'))
logger.addHandler(handler)

In [12]:
pathlib.Path('d', 'Applications', 'WebDriver')

WindowsPath('d/Applications/WebDriver')

In [19]:
chrome_options = Options()
# chrome_options.add_argument('--disable-extensions')
# chrome_options.add_argument('--disable-gpu')
# chrome_options.add_argument('--headless')
service = Service(executable_path=pathlib.WindowsPath('C:/Users/ISMEDocuments/chrome.exe'))
browser = webdriver.Chrome(service=service, options=chrome_options)

NoSuchDriverException: Message: Unable to obtain driver for chrome; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors/driver_location


In [ ]:
from selenium import webdriver
driver = webdriver.Chrome()
driver.get('https://www.nasa.gov')
headlines = driver.find_elements_by_class_name("headline")
for headline in headlines:
    print(headline.text.strip())
driver.close()
#C:\Users\ISME\Documents

In [ ]:
base_url = 'https://www.dns-shop.kz'
search_page = 'https://www.dns-shop.kz/catalog/17a892f816404e77/noutbuki/'

In [ ]:
laptops = {
    'id': [],
    'link': [],
    'descr_short': [],
    'price': []
}
browser.get(search_page)
soup = BeautifulSoup(browser.page_source, 'html.parser')
# Now we can search for keys needed
num_pages = int(soup.find_all('li', 'pagination-widget__page')[-1]['data-page-number'])
for page in range(1, num_pages+1):
    if page > 1:
        browser.get(search_page+f'?p={page}')
        sleep(5) # needed to give time to update prices as they seem to be loaded dynamically
        soup = BeautifulSoup(browser.page_source, 'html.parser')
    ids = soup.find_all('div', 'catalog-product ui-button-widget')
    descrs = soup.find_all('a', 'catalog-product__name ui-link ui-link_black')
    prices = soup.find_all('div', 'product-buy__price')
    print(f'Page: {page}: Got {len(ids)} ids, {len(descrs)} descriptions, {len(prices)} prices')
    for el in ids:
        laptops['id'].append(el['data-code'])
    for el in descrs:
        laptops['link'].append(el['href'])
        laptops['descr_short'].append(el.span.text)
    for el in prices:
        laptops['price'].append(el.text)

In [ ]:
for key, value in laptops.items():
    print(key, len(value))

In [ ]:
browser.quit()

In [ ]:
df = pd.DataFrame(laptops)
df.head()

In [ ]:
df['screen_size'] = df['descr_short'].apply(lambda x: re.findall(pattern=r'.+(?=\")', string=x)[0].strip())
df['resolution'] = df['descr_short'].apply(lambda x: re.findall(pattern=r'\d+x\d{,4}', string=x)[0].strip())
df['cpu'] = df['descr_short'].apply(lambda x: re.findall(pattern=r',[^,]+(?:intel|amd|apple).*?,', string=x, flags=re.I)[0][1:-1].strip())
#df['cpu_cores'] = df['descr_short'].apply(lambda x: re.findall(r'(?<=ядра:\s).*(?=\sГГц)', x, flags=re.I)[0].strip())
df['ram'] = df['descr_short'].apply(lambda x: re.findall(r'(?<=ram).*?(?=гб)', x, flags=re.I)[0].strip())
df['hdd_ssd'] = df['descr_short'].apply(lambda x: re.findall(r'(?<=hdd\s|ssd\s)\d+', x, flags=re.I)[0].strip())
df['gpu'] = df['descr_short'].apply(lambda x: x[x.index('[')+1 : -1].split(',')[-2].strip())
df['os'] = df['descr_short'].apply(lambda x: x[x.index('[')+1 : -1].split(',')[-1].strip())
df['price'] = df['price'].apply(lambda x: int(''.join(re.findall(r'\d+', x))))
df.tail()

In [ ]:
df.to_csv('datasets/laptops.csv', index=False)

TOPSIS CODE

In [ ]:
import numpy as np
print('numpy version:', np.__version__)

import pandas as pd
print('pandas version:', pd.__version__)

import matplotlib as mpl
print('matplotlib version:', mpl.__version__)
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('datasets/laptops.csv')
df.sample(8)

In [ ]:
# an example of short description
df.iat[60, 2]

In [ ]:
cpu_ratings = {
    'AMD Ryzen 9': 1,
    'Intel Core i9': 2,
    'Intel Core i7': 3,
    'Apple M2': 4,
    'AMD Ryzen 7': 5,
    'Intel Core i5': 6,
    'AMD Ryzen 5': 7,
    'Apple M1': 8,
    'AMD Ryzen 3': 9,
    'Intel Core i3': 10,
    'Intel Pentium': 11,
    'Intel Celeron': 12,
    'AMD Athlon': 13,
}
def map_cpu(value):
    for family, rating in cpu_ratings.items():
        if value.startswith(family):
            return rating
    return 99
df['cpu_rating'] = df['cpu'].apply(map_cpu)
df.sample(8)

In [ ]:
graphics_ratings = {
    'GeForce RTX 4090': 1,
    'GeForce RTX 4070': 2,
    'GeForce RTX 3080': 3,
    'GeForce RTX 3070': 4,
    'GeForce RTX 3060': 5,
    'GeForce RTX 3050': 6,
    'Apple M2': 6,
    'Apple M1': 7,
    'AMD Radeon': 7,
    'GeForce GTX 1660': 8,
    'GeForce GTX 1650': 9,
    'GeForce MX': 10,
    'Intel': 14,
}
def map_graphics(value):
    for family, rating in graphics_ratings.items():
        if value.startswith(family):
            return rating
    return 99
df['graphics_rating'] = df['gpu'].apply(map_graphics)
df.sample(8)

In [ ]:
# let's check if there are any 99 values
print(len(df[df['cpu_rating'] == 99]))
print(len(df[df['graphics_rating'] == 99]))

In [ ]:
# finally we will compare resolutions by their first values
df['res'] = df['resolution'].apply(lambda x: int(x[:x.index('x')]))
df.sample(8)

In [ ]:
# adjusting data types to float
df['screen_size'] = df['screen_size'].astype(float)
df['ram'] = df['ram'].astype(float)
df['hdd_ssd'] = df['hdd_ssd'].astype(float)
for col in df.select_dtypes(include=['int64']):
    df[col] = df[col].astype(float)
df.info()

TOPSIS Algorithm
Before starting calculations we have to:

obtain the weights of attributes indicating the relative significance of attribute to the decision maker. In our case let all the features be equally significant, so 
 
understand which attributes are benefit and which are cost. If ratings are translated as "the higher the better" then attribute is considered as benefit and otherwise it is considered as a cost. From our list 'price', 'cpu_rating' and 'graphics_rating' are cost attributes (note that we could use for example benchmark scores to make 'cpu_rating' and 'graphics_rating' benefit attributes). Other attributes are benefit

In [ ]:
performance_columns = {
    'price': 'cost',
    'screen_size': 'benefit',
    'ram': 'benefit',
    'hdd_ssd': 'benefit',
    'cpu_rating': 'cost',
    'graphics_rating': 'cost',
    'res': 'benefit'
}
W = np.ones((7,)) / len(performance_columns.keys())
W

In [ ]:
X = df[performance_columns.keys()].copy()
X

![alt text](image.png)

In [ ]:
Y = X / np.sqrt((X**2).sum(axis=0))
Y

In [ ]:
V = Y * W
V

3. Find positive and negative ideal solutions
By ideal solution we mean virtual variant having all minimum or maximum weighted values for attributes. Here we have to take into account if the attribute is benefit or cost.
So positive ideal solution will contain all maximum values from benefit attributes and minimum values from cost attributes, and vice versa for negative ideal solution

In [ ]:
pos_is = np.array([V[col].max() if col_type == 'benefit' else V[col].min() for col, col_type in performance_columns.items()])
neg_is = np.array([V[col].min() if col_type == 'benefit' else V[col].max() for col, col_type in performance_columns.items()])
print('Positive ideal solution:', pos_is)
print('Negative ideal solution:', neg_is)

![alt text](image-1.png)

In [ ]:
D_pos = np.sqrt(((V - pos_is)**2).sum(axis=1))
D_neg = np.sqrt(((V - neg_is)**2).sum(axis=1))
S = D_neg / (D_pos + D_neg)
best_alternative = np.argmax(S)
worst_alternative = np.argmin(S)
idxs = np.argsort(S.values)

In [ ]:
colors = ['#CC0000', '#FF3399', '#9933FF']

fig, ax = plt.subplots()
ax.scatter(D_pos, D_neg, alpha=0.5)
for i in range(1, 4):
    ax.scatter(D_pos.iloc[idxs[-i]], D_neg.iloc[idxs[-i]], color=colors[i-1], label=f'Top {i} TOPSIS score')
ax.scatter(D_pos.iloc[worst_alternative], D_neg.iloc[worst_alternative], color='k', label='Lowest TOPSIS score')
ax.set_xlabel('Distance to positive ideal solution')
ax.set_ylabel('Distance to negative ideal solution')
ax.set_title('Different laptop configurations in TOPSIS distances space')
ax.legend();

In [ ]:
print(df.iloc[best_alternative]['descr_short'])
print(f'Price: {df.iloc[best_alternative]["price"]}')

Modified TOPSIS Algorithm
Modified TOPSIS is similar to TOPSIS except the moment it uses weights. In original TOPSIS weights are applied to

In [ ]:
mod_pos_is = np.array([Y[col].max() if col_type == 'benefit' else Y[col].min() for col, col_type in performance_columns.items()])
mod_neg_is = np.array([Y[col].min() if col_type == 'benefit' else Y[col].max() for col, col_type in performance_columns.items()])
print('Positive ideal solution:', mod_pos_is)
print('Negative ideal solution:', mod_neg_is)

In [ ]:
mod_D_pos = np.sqrt((W*(V - pos_is)**2).sum(axis=1))
mod_D_neg = np.sqrt((W*(V - neg_is)**2).sum(axis=1))
mod_S = mod_D_neg / (mod_D_pos + mod_D_neg)
mod_best_alternative = np.argmax(mod_S)
mod_worst_alternative = np.argmin(mod_S)
mod_idxs = np.argsort(mod_S.values)

In [ ]:
fig, ax = plt.subplots()
ax.scatter(mod_D_pos, mod_D_neg, alpha=0.5)
for i in range(1, 4):
    ax.scatter(mod_D_pos.iloc[mod_idxs[-i]], mod_D_neg.iloc[mod_idxs[-i]], color=colors[i-1], label=f'Top {i} Modified TOPSIS score')
ax.scatter(mod_D_pos.iloc[mod_worst_alternative], mod_D_neg.iloc[mod_worst_alternative], color='k', label='Lowest Modified TOPSIS score')
ax.set_xlabel('Distance to positive ideal solution')
ax.set_ylabel('Distance to negative ideal solution')
ax.set_title('Different laptop configurations in TOPSIS distances space')
ax.legend();

We see that relative positions of data points were not changed but now we have them in different coordinate values. Remember that we only changed the moment weights are applied.
It might be interesting if relative positions will be influenced by switching algorithm from original TOPSIS to modified version in case when weights are not equal for features